In [ ]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("bittlingmayer/spelling")

print("Path to dataset files:", path)
corpus_filepath = os.path.join(path, "big.txt")
pickle_filepath='/kaggle/working/word_counts.pkl' #to save the word_counts dictionary(our frequency map) so we don't process the corpus everytime


import re
from collections import Counter, defaultdict
import os # To check if the file exists
import pickle # To save/load the Python object
import string # To get the alphabet easily

#Loading and preprocessing the corpus
def process_corpus(filepath='big.txt'):   #reading the corpus line by line, lowercasing, applying regular expressions, extracting words, counting frequncy, updating word_count, saving in pkl file, loading word counts, showing top 10 most common words and showing frequency of the and spelling
  """
  Reads a text file, processes it into lowercase words,
  """
  word_counts = Counter()
  try:
    with open(filepath, 'r', encoding='utf-8') as file:
      for line in file:
        words_in_line = re.findall(r'\b[a-zA-Z]+\b', line.lower())
        word_counts.update(words_in_line)
    print(f"Successfully processed the corpus.")
    print(f"Total words counted (including duplicates): {sum(word_counts.values())}")
    print(f"Unique words in vocabulary: {len(word_counts)}")
  except FileNotFoundError:
    print(f"Error: File not found at '{filepath}'.")
    print("Please ensure 'big.txt' is in the correct directory or update the path.")
    return Counter()
  except Exception as e:
    print(f"An error occurred: {e}")
    return Counter()
  return word_counts
    # word_counts is a dictionary(a counter object which is a type of dictionaries in python)
    #keys are words , values are the number of times each word appears


def save_word_counts(word_counts, pickle_path):
  """Saves the word_counts Counter object to a pickle file."""
  print(f"Saving word counts to {pickle_path}...")
  try:
    with open(pickle_path, 'wb') as file:
      pickle.dump(word_counts, file)
    print("Word counts saved successfully.")
  except Exception as e:
    print(f"Error saving word counts: {e}")


def load_word_counts(pickle_path):
  """Loads word counts from a pickle file."""
  print(f"Attempting to load word counts from {pickle_path}...")
  try:
    with open(pickle_path, 'rb') as file:
      word_counts = pickle.load(file)
    print("Word counts loaded successfully from pickle file.")
    print(f"Unique words in loaded vocabulary: {len(word_counts)}")
    return word_counts
  except FileNotFoundError:
    print("Pickle file not found.")
    return None # Indicate failure
  except Exception as e:
    print(f"Error loading word counts: {e}")
    return None # Indicate failure


WORD_COUNTS = None
if WORD_COUNTS is None:
  print("Processing the original corpus file...")
  WORD_COUNTS = process_corpus(corpus_filepath)


  if WORD_COUNTS:
    save_word_counts(WORD_COUNTS, pickle_filepath)
  else:
    print("Failed to process the corpus. Cannot save.")

if WORD_COUNTS:
  print("\n--- Sample Word Counts (from loaded or processed data) ---")
  # Display the 10 most common words
  print("Most common words:", WORD_COUNTS.most_common(10))
  # Check the count of a specific word
  print("Count for 'the':", WORD_COUNTS['the'])
  print("Count for 'spelling':", WORD_COUNTS['spelling'])
else:
  print("\nCould not load or process word counts. Cannot proceed.")

Path to dataset files: /kaggle/input/spelling
Processing the original corpus file...
Successfully processed the corpus.
Total words counted (including duplicates): 1100811
Unique words in vocabulary: 29056
Saving word counts to /kaggle/working/word_counts.pkl...
Error saving word counts: [Errno 2] No such file or directory: '/kaggle/working/word_counts.pkl'

--- Sample Word Counts (from loaded or processed data) ---
Most common words: [('the', 79809), ('of', 40024), ('and', 38312), ('to', 28765), ('in', 22023), ('a', 21124), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]
Count for 'the': 79809
Count for 'spelling': 4


In [ ]:
#Generating Candidate Corrections (Edit Distance 1 : Levenshtein distance)
def edits1(word):
  """
  Generates all possible strings that are one edit away from 'word'.
  Args:
    word (str): The input word.

  Returns:
    set: A set of all unique strings one edit away from 'word'.
  """
  letters    = string.ascii_lowercase
  splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]

  # Deletes: Remove one character from the right part
  deletes    = [L + R[1:]               for L, R in splits if R] # Check 'if R' ensures right part isn't empty

  # Transposes: Swap adjacent characters in the right part (if long enough)
  transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1] # Need at least 2 chars to swap

  # Replaces: Replace the first char of the right part with any letter
  replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters] # Check 'if R'

  # Inserts: Insert any letter between the left and right parts
  inserts    = [L + c + R               for L, R in splits for c in letters]

  # Combine all edits into a set to get unique candidates
  return set(deletes + transposes + replaces + inserts)




def known(words_to_check, vocabulary):
  """
  Filters a collection of words, returning only those present in the vocabulary.
  Returns:
    set: A subset of `words_to_check` containing only words found in the vocabulary.
  """
  return set(w for w in words_to_check if w in vocabulary)



def suggest_correction_v1(word, vocabulary):
    """
    Suggests the most probable spelling correction for `word` based on edit distance 1.

    Returns:
        str: The suggested correction (most frequent known word at edit distance 1),
             or the original word if it's known or no corrections are found.
    """
    # Normalize the input word to lowercase
    word = word.lower()

    # 1. Check if the word itself is correct in the vocabulary
    if word in vocabulary:
        print(f"'{word}' is already in the vocabulary.")
        return word

    # 2. Generate all possible words with one edit distance.
    candidates_edits1 = edits1(word)
    print(f"Generated {len(candidates_edits1)} candidates at edit distance 1 for '{word}'.")

    # 3. Filter these candidates to find those that are known words in our vocabulary.
    known_candidates = known(candidates_edits1, vocabulary)
    print(f"Found {len(known_candidates)} known candidates: {known_candidates or '{}'}")

    # 4. If there are known candidates, choose the one with the highest frequency.
    if known_candidates:
        best_candidate = max(known_candidates, key=vocabulary.get)
        print(f"Best candidate based on frequency: '{best_candidate}' (Freq: {vocabulary.get(best_candidate)})")
        return best_candidate

    return word  # If no corrections are found, return the original word



if WORD_COUNTS:
  print("\n--- Testing Spell Correction (Edit Distance 1) ---")
  test_words = ["speling", "babi", "hapy", "korrectud", "thaa", "receeve", "apple","inglis"]

  for test_word in test_words:
    correction = suggest_correction_v1(test_word, WORD_COUNTS)
    print(f"Original: '{test_word}', Suggested: '{correction}'")
    print("-" * 20)
else:
  print("\nCannot run spell correction tests as WORD_COUNTS is not available.")


--- Testing Spell Correction (Edit Distance 1) ---
Generated 390 candidates at edit distance 1 for 'speling'.
Found 1 known candidates: {'spelling'}
Best candidate based on frequency: 'spelling' (Freq: 4)
Original: 'speling', Suggested: 'spelling'
--------------------
Generated 234 candidates at edit distance 1 for 'babi'.
Found 2 known candidates: {'babe', 'baby'}
Best candidate based on frequency: 'baby' (Freq: 45)
Original: 'babi', Suggested: 'baby'
--------------------
Generated 234 candidates at edit distance 1 for 'hapy'.
Found 2 known candidates: {'hay', 'happy'}
Best candidate based on frequency: 'happy' (Freq: 218)
Original: 'hapy', Suggested: 'happy'
--------------------
Generated 492 candidates at edit distance 1 for 'korrectud'.
Found 0 known candidates: {}
Original: 'korrectud', Suggested: 'korrectud'
--------------------
Generated 232 candidates at edit distance 1 for 'thaa'.
Found 3 known candidates: {'thaw', 'than', 'that'}
Best candidate based on frequency: 'that' (Fr

In [ ]:
#we can notice that this approach can handle one typo ( given that the word is in our corpus) more than one typo like(korrectud) doesnt work. Which leads us to the approach of check distance2.
#Generating Candidate Corrections (Edit Distance 2 : Levenshtein distance)

def edits2(word):
  # Apply edits1 to each result of edits1(word)
  return set(e2 for e1 in edits1(word) for e2 in edits1(e1))


def suggest_correction_v2(word, vocabulary):
  """
  Returns:
    str: The suggested correction, prioritizing lower edit distance and then
         highest frequency. Returns the original word if no correction found.
  """
  word = word.lower()

  #Priority 1: the word in vocab
  if word in vocabulary:
    return word

  # Priority 2: Candidates at Edit Distance 1
  candidates_edits1 = edits1(word)
  known_candidates_edits1 = known(candidates_edits1, vocabulary)
  if known_candidates_edits1:
    best_candidate = max(known_candidates_edits1, key=vocabulary.get)
    return best_candidate

  # Priority 3: Candidates at Edit Distance 2
  candidates_edits2 = edits2(word)
  known_candidates_edits2 = known(candidates_edits2, vocabulary)

  if known_candidates_edits2:
    best_candidate = max(known_candidates_edits2, key=vocabulary.get)
    return best_candidate

  # Priority 4: No known correction found
  return word



if WORD_COUNTS:
  print("\n--- Testing Spell Correction (Edit Distance 1 & 2) ---")
  # Include words that might need 2 edits, and some previous ones
  test_words = ["speling",
                "korrectud",
                "hapy",
                "algorithim",
                "receeve",
                "appel",
                "language",
                "compromissed",
                "activiti",
                "inglis",
               ]

  for test_word in test_words:
    print(f"Original: '{test_word}'")
    correction = suggest_correction_v2(test_word, WORD_COUNTS)
    print(f"Suggested: '{correction}'")
    print("-" * 20)
else:
  print("\nCannot run spell correction tests as WORD_COUNTS is not available.")




def tokenize_sentence(sentence):
  """
  Splits a sentence into words and punctuation/whitespace sequences.
  Example: "Helo worl!" -> ['Helo', ' ', 'worl', '!']
  """
  # This regex finds sequences of letters OR sequences of non-letters
  return re.findall(r'[a-zA-Z]+|[^a-zA-Z\s]+|\s+', sentence)


#Full sentence-level corrections
def correct_sentence_simple(sentence, vocabulary):
  """
  Corrects a sentence by applying suggest_correction_v2 to each word.

  Returns:
    str: The corrected sentence.
  """
  tokens = tokenize_sentence(sentence)
  corrected_tokens = []

  for token in tokens:
    if re.match(r'[a-zA-Z]+', token):
      original_case = token
      correction = suggest_correction_v2(token.lower(), vocabulary)


      if original_case.istitle() and correction:
          correction = correction.title()
      elif original_case.isupper() and len(original_case) > 1 and correction: # Avoid capitalizing single-letter words like 'a'
           # Check if correction is not empty
           correction = correction.upper()
      # Otherwise, keep the correction lowercase (or as returned)

      corrected_tokens.append(correction)
    else:
      # If it's not a word (punctuation, space), keep it as is
      corrected_tokens.append(token)

  # Join the tokens back together
  return "".join(corrected_tokens)


if WORD_COUNTS:
  print("\n--- Testing Sentence Correction ---")
  test_sentences = [
      "Thiss sentense has somme spelling mistaks.",
      "Wat is the korrectud anser?",
      "I receeved the mesage.",
      "All is well.",
      "ACTIVITI level is high.",
      "It was hear.",
      " i am so hapgy"
  ]

  for sentence in test_sentences:
    corrected = correct_sentence_simple(sentence, WORD_COUNTS)
    print(f"Original:  '{sentence}'")
    print(f"Corrected: '{corrected}'")
    print("-" * 30)
else:
  print("\nCannot run sentence correction tests as WORD_COUNTS is not available.")






--- Testing Spell Correction (Edit Distance 1 & 2) ---
Original: 'speling'
Suggested: 'spelling'
--------------------
Original: 'korrectud'
Suggested: 'corrected'
--------------------
Original: 'hapy'
Suggested: 'happy'
--------------------
Original: 'algorithim'
Suggested: 'algorithim'
--------------------
Original: 'receeve'
Suggested: 'receive'
--------------------
Original: 'appel'
Suggested: 'appeal'
--------------------
Original: 'language'
Suggested: 'language'
--------------------
Original: 'compromissed'
Suggested: 'compromised'
--------------------
Original: 'activiti'
Suggested: 'activity'
--------------------
Original: 'inglis'
Suggested: 'english'
--------------------

--- Testing Sentence Correction ---
Original:  'Thiss sentense has somme spelling mistaks.'
Corrected: 'This sentence has some spelling mistake.'
------------------------------
Original:  'Wat is the korrectud anser?'
Corrected: 'Wat is the corrected answer?'
------------------------------
Original:  'I rec

In [ ]:
#N-GRAM¶
#Generating Bigram counts

def process_corpus_ngrams(filepath='big.txt'):
  """
  Reads a text file, processes it into lowercase words, and returns
  frequency counts for both unigrams (words) and bigrams (word pairs).

  Returns:
    tuple: (unigram_counts, bigram_counts)
           unigram_counts: collections.Counter {word: count}
           bigram_counts: collections.defaultdict(Counter) {prev_word: {word: count}}
           Returns (None, None) if file processing fails.
  """
  print(f"Attempting to read corpus file for N-grams: {filepath}")
  unigram_counts = Counter()

  bigram_counts = defaultdict(Counter)
  total_words = 0
  processed_words = 0

  try:
    with open(filepath, 'r', encoding='utf-8') as file:
      prev_word = None
      for line in file:

        words_in_line = re.findall(r'\b[a-z]+\b', line.lower())

        if not words_in_line:
          prev_word = None # Reset if line is empty or has no words
          continue

        for word in words_in_line:
          unigram_counts[word] += 1
          processed_words += 1
          if prev_word is not None:
              # Increment count for the bigram (prev_word, word)
              bigram_counts[prev_word][word] += 1

          prev_word = word # Update prev_word for the next iteration


    total_words = sum(unigram_counts.values())
    print(f"Successfully processed the corpus for N-grams.")
    print(f"Total words counted (unigrams): {total_words}")
    print(f"Unique words in vocabulary: {len(unigram_counts)}")
    # Calculate total number of bigram pairs counted
    total_bigrams = sum(sum(inner_counter.values()) for inner_counter in bigram_counts.values())
    print(f"Total bigram pairs counted: {total_bigrams}")
    # Calculate unique preceding words in bigrams
    print(f"Unique preceding words in bigrams: {len(bigram_counts)}")


  except FileNotFoundError:
    print(f"Error: File not found at '{filepath}'. Cannot process.")
    return None, None
  except Exception as e:
    print(f"An error occurred during N-gram processing: {e}")
    return None, None

  return unigram_counts, bigram_counts, total_words


def save_ngram_data(data, pickle_path):
  """Saves (unigrams, bigrams, total_words) tuple to a pickle file."""
  print(f"Saving N-gram data to {pickle_path}...")
  try:
    with open(pickle_path, 'wb') as file:
      pickle.dump(data, file)
    print("N-gram data saved successfully.")
  except Exception as e:
    print(f"Error saving N-gram data: {e}")

def load_ngram_data(pickle_path):
  """Loads (unigrams, bigrams, total_words) tuple from a pickle file."""
  print(f"Attempting to load N-gram data from {pickle_path}...")
  try:
    with open(pickle_path, 'rb') as file:
      unigrams, bigrams, total_words = pickle.load(file)
    print("N-gram data loaded successfully.")
    print(f"Unique words loaded: {len(unigrams)}")
    print(f"Unique preceding words loaded: {len(bigrams)}")
    print(f"Total word count loaded: {total_words}")
    return unigrams, bigrams, total_words
  except FileNotFoundError:
    print("N-gram pickle file not found.")
    return None, None, None
  except Exception as e:
    print(f"Error loading N-gram data: {e}")
    return None, None, None


ngram_pickle_filepath = '/kaggle/working/ngram_data.pkl'

previous_output_ngram_path = '/kaggle/input/spelling/ngram_data.pkl'
UNI_COUNTS, BI_COUNTS, TOTAL_WORDS = None, None, None

# List of paths to try loading from
load_paths_ngram = [previous_output_ngram_path, ngram_pickle_filepath]

# 1. Try loading from potential previous output locations
found_data = False
for path in load_paths_ngram:
    if os.path.exists(path):
        UNI_COUNTS, BI_COUNTS, TOTAL_WORDS = load_ngram_data(path)
        if UNI_COUNTS is not None: # Check if loading was truly successful
            found_data = True
            break # Stop searching once loaded

# 2. If loading failed, process the original corpus
if not found_data:
  print("\n--- Loading Failed: Processing Corpus for N-grams ---")
  if not os.path.exists(corpus_filepath):
      print(f"ERROR: Corpus file not found at {corpus_filepath}. Please check the path.")
  else:
      UNI_COUNTS, BI_COUNTS, TOTAL_WORDS = process_corpus_ngrams(corpus_filepath)

  # 3. If processing was successful, save the results
  if UNI_COUNTS and BI_COUNTS:
    os.makedirs('/kaggle/working', exist_ok=True)  # Ensure directory exists
    save_ngram_data((UNI_COUNTS, BI_COUNTS, TOTAL_WORDS), ngram_pickle_filepath)

  else:
    print("Failed to process the corpus for N-grams. Cannot save.")





--- Loading Failed: Processing Corpus for N-grams ---
Attempting to read corpus file for N-grams: /kaggle/input/spelling/big.txt
Successfully processed the corpus for N-grams.
Total words counted (unigrams): 1100811
Unique words in vocabulary: 29056
Total bigram pairs counted: 1079057
Unique preceding words in bigrams: 28730
Saving N-gram data to /kaggle/working/ngram_data.pkl...
N-gram data saved successfully.


In [ ]:
#Calculating bigram probabilities
def P_unigram(word, unigram_counts, total_words, vocab_size):
  """Calculates smoothed Unigram probability P(word)."""
  # Add-1 Smoothing
  count = unigram_counts.get(word, 0)
  return (count + 1) / (total_words + vocab_size)

def P_bigram(word, prev_word, unigram_counts, bigram_counts, vocab_size):
  """Calculates smoothed Bigram probability P(word | prev_word)."""
  bigram_count = bigram_counts.get(prev_word, Counter()).get(word, 0)
  prev_unigram_count = unigram_counts.get(prev_word, 0)

  # V is the size of the vocabulary
  return (bigram_count + 1) / (prev_unigram_count + vocab_size)


VOCAB_SIZE = 0
if UNI_COUNTS:
    VOCAB_SIZE = len(UNI_COUNTS)
    print(f"Vocabulary Size (for smoothing): {VOCAB_SIZE}")



from math import log

def suggest_correction_ngram_comparative(word_to_correct, prev_word, next_word,
                                        unigram_counts, bigram_counts, total_words, vocab_size):
    """
    Spelling correction optimized for big.txt corpus, using context-aware n-gram scoring.
    """
    original_word = word_to_correct.lower()

    # 1. Candidate generation (similar to Norvig's but with context awareness)
    candidates = (known([original_word], unigram_counts) or
                 known(edits1(original_word), unigram_counts) or
                 known(edits2(original_word), unigram_counts) or
                 [original_word])

    # 2. Scoring with context (optimized for big.txt patterns)
    def get_score(candidate):
        score = 0.0

        # Base unigram score (log probability)
        unigram_prob = P_unigram(candidate, unigram_counts, total_words, vocab_size)
        score += log(unigram_prob + 1e-10)

        # Previous context bonus
        if prev_word and (prev_word, candidate) in bigram_counts:
            bigram_prob = P_bigram(candidate, prev_word, unigram_counts, bigram_counts, vocab_size)
            score += log(bigram_prob + 1e-10) * 1.5  # Weight context more heavily

        # Next context bonus
        if next_word and (candidate, next_word) in bigram_counts:
            bigram_prob = P_bigram(next_word, candidate, unigram_counts, bigram_counts, vocab_size)
            score += log(bigram_prob + 1e-10) * 1.5

        return score

    # 3. Select best candidate with Norvig-inspired thresholds
    best_candidate = max(candidates, key=get_score)
    best_score = get_score(best_candidate)

    original_score = get_score(original_word) if original_word in candidates else -float('inf')

    # Only correct if significant improvement
    if (best_candidate != original_word and
        (best_score - original_score) > log(20)):
        return best_candidate

    return original_word

def correct_sentence_ngram_comparative(sentence, unigram_counts, bigram_counts, total_words, vocab_size):
    """Sentence correction optimized for big.txt content patterns."""
    tokens = tokenize_sentence(sentence)
    corrected_tokens = []

    for i, token in enumerate(tokens):
        if not re.match(r'[a-zA-Z\']+', token):
            corrected_tokens.append(token)
            continue

        original_case = token
        original_word = token.lower()

        # Get context words (skip non-words)
        prev_word = corrected_tokens[-1].lower() if (corrected_tokens and
                          re.match(r'[a-zA-Z\']+', corrected_tokens[-1])) else None
        next_word = tokens[i+1].lower() if (i+1 < len(tokens) and
                          re.match(r'[a-zA-Z\']+', tokens[i+1])) else None

        correction = suggest_correction_ngram_comparative(
            original_word, prev_word, next_word,
            unigram_counts, bigram_counts, total_words, vocab_size
        )

        # Case restoration
        if original_case.istitle():
            correction = correction.title()
        elif original_case.isupper():
            correction = correction.upper()
        elif "'" in original_case:  # Handle "I'M" -> "I'm" etc.
            parts = original_case.split("'")
            if len(parts) == 2 and len(parts[1]) > 0:
                correction = correction.replace("'", "'").replace(
                    correction.split("'")[1].lower(), parts[1].lower())

        corrected_tokens.append(correction)

    return "".join(corrected_tokens)



if __name__ == "__main__" and 'UNI_COUNTS' in globals() and 'BI_COUNTS' in globals():
    print("\n--- Testing with big.txt-optimized Correction ---")

    # These examples reflect common errors that should be correctable with big.txt content
    test_sentences = [
        "I can not beleive it.",
        "You cay thaaat there was a gentleman.",
        "Their is a problem here.",
        "She recieved the letter.",
        "Its all about the context.",
        "The goverment announced plans.",
        "He completly forgot.",
        "They where happy together.",
        "This is more prefered.",
        "An artifical intelligence.",
        "Tha definate answer.",
        "A seperate occasion.",
        "hiz eyes are beautifol.",
    ]

    for sentence in test_sentences:
        corrected = correct_sentence_ngram_comparative(
            sentence, UNI_COUNTS, BI_COUNTS, TOTAL_WORDS, VOCAB_SIZE
        )
        print(f"Original:  '{sentence}'")
        print(f"Corrected: '{corrected}'")
        print("-" * 50)

Vocabulary Size (for smoothing): 29056

--- Testing with big.txt-optimized Correction ---
Original:  'I can not beleive it.'
Corrected: 'I can not believe it.'
--------------------------------------------------
Original:  'You cay thaaat there was a gentleman.'
Corrected: 'You may that there was a gentleman.'
--------------------------------------------------
Original:  'Their is a problem here.'
Corrected: 'Their is a problem here.'
--------------------------------------------------
Original:  'She recieved the letter.'
Corrected: 'She received the letter.'
--------------------------------------------------
Original:  'Its all about the context.'
Corrected: 'Its all about the context.'
--------------------------------------------------
Original:  'The goverment announced plans.'
Corrected: 'The government announced plans.'
--------------------------------------------------
Original:  'He completly forgot.'
Corrected: 'He completely forgot.'
--------------------------------------------

In [ ]:
#GUI
from IPython.display import display
import ipywidgets as widgets

# --- GUI Widgets ---
input_text = widgets.Textarea(
    value="Their is a probleem with this sentance.",
    placeholder='Type a sentence with spelling mistakes',
    description='Input:',
    layout={'width': '600px', 'height': '100px'}
)

output_text = widgets.Textarea(
    value="",
    description='Corrected:',
    disabled=True,
    layout={'width': '600px', 'height': '100px'}
)

button = widgets.Button(description="Correct Spelling")

def on_button_click(b):
    corrected = correct_sentence_ngram_comparative(
        input_text.value,
        UNI_COUNTS,
        BI_COUNTS,
        TOTAL_WORDS,
        VOCAB_SIZE
    )
    output_text.value = corrected

button.on_click(on_button_click)

display(widgets.VBox([input_text, button, output_text]))


In [ ]:
testfile_filepath='/kaggle/input/spelling/spell-testset1.txt'

test_data = {}

with open(testfile_filepath, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        correct, misspellings = line.split(':')
        correct = correct.strip()
        misspellings_list = misspellings.strip().split()
        test_data[correct] = misspellings_list

import re

def tokenize(text):
    """Simple tokenizer to split into words, preserving apostrophes."""
    return re.findall(r"[a-zA-Z']+", text.lower())

def evaluate_corrections(test_data, unigram_counts, bigram_counts, total_words, vocab_size):
    TP, FP, FN, TN = 0, 0, 0, 0

    for correct_word, misspellings in test_data.items():
        for original_word in misspellings:
            corrected_word = correct_sentence_ngram_comparative(
                original_word, unigram_counts, bigram_counts, total_words, vocab_size
            )

            orig_tokens = tokenize(original_word)
            corr_tokens = tokenize(corrected_word)
            true_tokens = tokenize(correct_word)

            for orig_word, corr_word, true_word in zip(orig_tokens, corr_tokens, true_tokens):
                if orig_word != true_word:  # word misspelled originally
                    if corr_word == true_word:  # corrected correctly
                        TP += 1
                    else:
                        FN += 1
                else:  # word was correct originally
                    if corr_word == true_word:
                        TN += 1
                    else:
                        FP += 1

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    accuracy = (TP + TN) / (TP + FP + FN + TN) if (TP + FP + FN + TN) > 0 else 0

    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"Accuracy:  {accuracy:.4f}")

    return precision, recall, accuracy

precision, recall, accuracy = evaluate_corrections(
    test_data, UNI_COUNTS, BI_COUNTS, TOTAL_WORDS, VOCAB_SIZE
)


Precision: 1.0000
Recall:    0.7481
Accuracy:  0.7481
